#### Here we investigate the relashionship between:
    - mRNA level predictability of a landmark gene 
    and 
    - its known organelle level biological function using GO annotations

In [ ]:
import pandas as pd
import numpy as np
def locations_of_substring(string, substring):
    """Return a list of locations of a substring."""
    substring_length = len(substring)    
    def recurse(locations_found, start):
        location = string.find(substring, start)
        if location != -1:
            return recurse(locations_found + [location], location+substring_length)
        else:
            return locations_found
    return recurse([], 0)

#### For LUAD dataset:

1 - Read predictability map of categorical features (using MLP model)

2 - Assign the feature categories to compartments/stains

3 - Read functional annotations of the reference set according to DAVIDs output and add columns for each channel
   - Add channel specific annotation to each columns channel
   

In [ ]:
import sys
sys.path.insert(0, '../utils/') 
from readProfiles import rename_affyprobe_to_genename
from saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
########### 1 ###########
filename='../results/SingleGenePred_cpCategoryMap/cat_scores_maps.xlsx'
saved_scores=pd.read_excel(filename, sheet_name=None)
# which_ds_model='LUAD-9-MLP-ht'
which_ds_model='LUAD-9-MLP-keras-ht'
# which_ds_model='LUAD-9-lasso-ht'
dfcats=saved_scores[which_ds_model].rename(columns={'Unnamed: 0':'ID'})
dfcats=dfcats[dfcats.columns[~dfcats.isna().any()].tolist()]
dfcats2,_=rename_affyprobe_to_genename(dfcats.set_index('ID').T,dfcats.ID.tolist())
dfcats=dfcats2.T.reset_index()


########### 2 ###########
Channelss=['DNA','RNA','AGP','Mito','ER']
Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape|Cells_AreaShape','Mito','ER']
# Channelss_cats=['DNA|Nuclei_AreaShape','RNA','AGP|Cytoplasm_AreaShape','Mito','ER']

for ci in range(len(Channelss)):
    dfcats['max_'+Channelss[ci]]=dfcats.loc[:,dfcats.columns.str.contains(Channelss_cats[ci])].max(axis=1)

    
dfcats['top_channel']=dfcats[['max_'+Channelss[ci] for ci in range(len(Channelss))]].idxmax(axis="columns")
########### 3 ###########
# gene_cats_bpcc=pd.read_csv('./go_BP_CC_MF_DIRECT_921.txt',delimiter='\t')
gene_cats_bpcc=pd.read_csv('./source/GO_bp_cc_mf_direct_LUAD_976.txt',delimiter='\t')
comps=['mitochondri','golgi','membrane','cytoskeleton','actin','endoplasmic','rna','nucleol',\
       'cell division','mitosis','mitotic','cell cycle']

# GOTERM_BP_DIRECT
# GOTERM_CC_DIRECT
# GOTERM_MF_DIRECT
for c in comps:
    gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_MF_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('go')[0] for si in locations_of_substring(x,c)]) if c in x else '')#+\
#     gene_cats_bpcc['UP_KW_BIOLOGICAL_PROCESS'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
#     gene_cats_bpcc['UP_KW_CELLULAR_COMPONENT'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
#     gene_cats_bpcc['UP_KW_MOLECULAR_FUNCTION'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split('~')[-1]+x[si:].split('kw')[0] for si in locations_of_substring(x,c)]) if c in x else '')#+\
# #     gene_cats_bpcc['UP_SEQ_FEATURE'].astype(str).str.lower().apply(lambda x:\
#     ''.join([x[:si].split(':')[-1]+','+x[si:].split(',')[0] for si in locations_of_substring(x,c)]) if c in x else '')
    
gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc['rna']+gene_cats_bpcc['nucleol']
gene_cats_bpcc['DNA']=gene_cats_bpcc['cell division']+gene_cats_bpcc['mitosis']+\
gene_cats_bpcc['mitotic']+gene_cats_bpcc['cell cycle']

gene_cats_bpcc['cytoskeleton-actin']=gene_cats_bpcc['cytoskeleton']+gene_cats_bpcc['actin']+\
gene_cats_bpcc['golgi']+gene_cats_bpcc['membrane']
# gene_cats_bpcc['ER']=gene_cats_bpcc['endoplasmic']
gene_cats_bpcc['mitochondria']=gene_cats_bpcc['mitochondri']

gene_cats_bpcc=pd.merge(gene_cats_bpcc,dfcats,how='inner',on='ID')

#########################
Channelss_dict={'DNA':'DNA','RNA_nucleoli':'RNA','cytoskeleton-actin':'AGP','mitochondria':'Mito','endoplasmic':'ER'}
Chan_rev_dict = dict(zip(Channelss_dict.values(),Channelss_dict.keys()))

comps2=['mitochondri','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
gene_cats_bpcc['any_comps']=''
for co in comps2:
    gene_cats_bpcc['any_comps']=gene_cats_bpcc['any_comps']+gene_cats_bpcc[co]
        
        
        
# gene_cats_bpcc=gene_cats_bpcc[gene_cats_bpcc['any_comps']!=''].reset_index(drop=True)        
from sklearn.metrics import confusion_matrix
from scipy.stats import fisher_exact
# top_bool=(gene_cats_bpcc['top58']==True).values
table2=pd.DataFrame(index=Channelss_dict.keys(),columns=Channelss_dict.values())
table3=pd.DataFrame(index=Channelss,columns=['odds ratio','med_restComp_oddsratio','restComp_oddsratio','anyComp_oddsratio','top-ratio'])
# table3=pd.DataFrame(index=Channelss,columns=['Prevalence','anyComp-Prevalence','noComp-Prevalence','top-ratio'])

table=[]
for c in Channelss:
    print(c)
    c_r=list(set(Channelss)-set([c]))
#     gene_cats_bpcc_highP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]>0.7)]
#     gene_cats_bpcc_lowP=gene_cats_bpcc[(gene_cats_bpcc['max_'+c]<0.1)]
    
    low_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[:100].index
    high_ind=gene_cats_bpcc[['max_'+c]].sort_values(by='max_'+c)[-100:].index    
    gene_cats_bpcc_highP=gene_cats_bpcc.loc[high_ind].reset_index(drop=True)
    gene_cats_bpcc_lowP=gene_cats_bpcc.loc[low_ind].reset_index(drop=True)
    gene_cats_bpcc2=gene_cats_bpcc.copy()
    
#     top_bool=(gene_cats_bpcc['max_'+c]>.6).values
    top_bool=(gene_cats_bpcc['max_'+c]<0).values
    
    print(np.sum(gene_cats_bpcc.loc[gene_cats_bpcc['max_'+c]<0,['max_'+Channelss[ci] for ci in range(len(Channelss))]].max(axis=1)>0.3))
    n_top=sum(top_bool)
    print(n_top)
    
    comps2=['mitochondria','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
    table1=pd.DataFrame(index=comps2+['any comp','no comp'],columns=['Prevalence','p-value','odds ratio'])
    for co in comps2:
        
        enr_ratio=gene_cats_bpcc2[top_bool & (gene_cats_bpcc2[co]!='')].shape[0]/\
        gene_cats_bpcc2[(gene_cats_bpcc2[co]!='')].shape[0]
        comp_bool=(gene_cats_bpcc2[co]!='').values
        oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, comp_bool))
        
        table1.loc[co,['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio
#         print(co,": ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
        table2.loc[co,c]=oddsratio
         
    
    enr_ratio=gene_cats_bpcc[top_bool & (gene_cats_bpcc['any_comps']!='')].shape[0]/\
    gene_cats_bpcc[(gene_cats_bpcc['any_comps']!='')].shape[0]
    any_oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']!='').values))
#     print("any_comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
    table1.loc['any comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,any_oddsratio
    print(any_oddsratio)

    nocomp_enr_ratio=gene_cats_bpcc[(top_bool) & (gene_cats_bpcc['any_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']=='')].shape[0]

    comps2=['mitochondria','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
    comps2.remove(Chan_rev_dict[c])
    
    med_restComp_oddsratio=table2.loc[comps2,c].median()
    gene_cats_bpcc['rest_comps']=''
    for co in comps2:
        gene_cats_bpcc['rest_comps']=gene_cats_bpcc['rest_comps']+gene_cats_bpcc[co]    
    
    rest_enr_ratio=gene_cats_bpcc[(top_bool) & (gene_cats_bpcc['rest_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['rest_comps']=='')].shape[0]    
    rest_oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['rest_comps']!='').values))
    
    table.append(table1)
#     print('num top ('+ str(n_top)+')/total genes (912): ', np.round((n_top/912)*100,2),'%')
    table3.loc[c,['odds ratio','med_restComp_oddsratio','restComp_oddsratio','anyComp_oddsratio','top-ratio']]=\
    table1.loc[Chan_rev_dict[c],'odds ratio'],med_restComp_oddsratio,rest_oddsratio,\
    any_oddsratio,np.round((n_top/912)*100,2)
    
#     table3.loc[c,['Prevalence','anyComp-Prevalence','noComp-Prevalence','top-ratio']]=table1.loc[Chan_rev_dict[c],'Prevalence'],\
#     enr_ratio*100,nocomp_enr_ratio*100,np.round((n_top/912)*100,2)    
    
#     print(Chan_rev_dict[c],': ',table1.loc[Chan_rev_dict[c],['Prevalence']].values)
    
# # table3['dif']=table3['Prevalence']-table3['anyComp-Prevalence']
table3['dif']=table3['odds ratio']-table3['restComp_oddsratio']
table3['dif2']=table3['odds ratio']-table3['med_restComp_oddsratio']
table3['dif3']=table3['odds ratio']-table3['anyComp_oddsratio']
# print(table3['dif'].min(),table3['dif'].sum())
# table3

source_data_add='../results/Figs_Source_Data.xlsx'
if 0:
    saveAsNewSheetToExistingFile(source_data_add,pd.concat([table3.astype(float).round(3)[['odds ratio','restComp_oddsratio']],x1],axis=1),'ExtendedData5')

## GO terms search for overlap of highly predictable genes (top 58)

In [ ]:
# top58=pd.read_csv('./top_58_common.txt',header=None)[0].tolist()
# gene_cats_bpcc=pd.read_csv('./go_bp_cc_D2021_each_gene_cat.txt',delimiter='\t')
import sys
sys.path.insert(0, '../utils/') 
from readProfiles import rename_affyprobe_to_genename
from saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile
top58=pd.read_csv('./source/top_59_atleast_topIn3.txt',header=None)[0].tolist()
gene_cats_bpcc=pd.read_csv('./source/GO_bp_cc_mf_direct_intersection_782.txt',delimiter='\t')

# comps=['mitochondri','Golgi','membrane','cytoskeleton','actin','endoplasmic','RNA','nucleol','cell division','mitosis','mitotic','cell cycle']

# for c in comps:
#     gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).apply(lambda x: x if c in x else '')+\
#     gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).apply(lambda x: x[:x.find(c)].split('~')[-1]+x[x.find(c):].split('GO')[0] if c in x else '')
comps=['mitochondri','golgi','membrane','cytoskeleton','actin','endoplasmic','rna','nucleol',\
       'cell division','mitosis','mitotic','cell cycle','cytokine','hormone']

for c in comps:
    gene_cats_bpcc[c]=gene_cats_bpcc['GOTERM_BP_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('GO')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_CC_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('GO')[0] for si in locations_of_substring(x,c)]) if c in x else '')+\
    gene_cats_bpcc['GOTERM_MF_DIRECT'].astype(str).str.lower().apply(lambda x:\
    ''.join([x[:si].split('~')[-1]+x[si:].split('GO')[0] for si in locations_of_substring(x,c)]) if c in x else '')
    
gene_cats_bpcc['RNA_nucleoli']=gene_cats_bpcc['rna']+gene_cats_bpcc['nucleol']
gene_cats_bpcc['DNA']=gene_cats_bpcc['cell division']+gene_cats_bpcc['mitosis']+gene_cats_bpcc['mitotic']+gene_cats_bpcc['cell cycle']

gene_cats_bpcc['cytoskeleton-actin']=gene_cats_bpcc['cytoskeleton']+gene_cats_bpcc['actin']

gene_cats_bpcc.loc[gene_cats_bpcc['ID'].isin(top58),'top58']=True



#####################################
comps2=['mitochondri','golgi','membrane','cytoskeleton-actin','endoplasmic','RNA_nucleoli','DNA']
from sklearn.metrics import confusion_matrix
from scipy.stats import fisher_exact
top_bool=(gene_cats_bpcc['top58']==True).values

table1=pd.DataFrame(index=comps2+['any comp','no comp'],columns=['Prevalence','p-value','odds ratio'])
for co in comps2:
    enr_ratio=gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc[co]!='')].shape[0]/\
    gene_cats_bpcc[(gene_cats_bpcc[co]!='')].shape[0]
    comp_bool=(gene_cats_bpcc[co]!='').values
    oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, comp_bool))
#     print(co, ':',gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc[co]!='')].shape[0],\
#          ', ',gene_cats_bpcc[(gene_cats_bpcc[co]!='')].shape[0])

    table1.loc[co,['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio
    print(co,": ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
    
gene_cats_bpcc['any_comps']=''
for co in comps2:
    gene_cats_bpcc['any_comps']=gene_cats_bpcc['any_comps']+gene_cats_bpcc[co]
     
    
enr_ratio=gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc['any_comps']!='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']!='')].shape[0]
oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']!='').values))
print("any_comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
table1.loc['any comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio


enr_ratio=gene_cats_bpcc[(gene_cats_bpcc['top58']==True) & (gene_cats_bpcc['any_comps']=='')].shape[0]/\
gene_cats_bpcc[(gene_cats_bpcc['any_comps']=='')].shape[0]
oddsratio, pvalue = fisher_exact(confusion_matrix(top_bool, (gene_cats_bpcc['any_comps']=='').values))
print("no comps: ",np.round(enr_ratio*100,2),'%     ','pvalue:', np.round(pvalue,2),'  oddsratio:',np.round(oddsratio,2))
table1.loc['no comp',['Prevalence','p-value','odds ratio']]=enr_ratio*100,pvalue,oddsratio

# print('num top (52)/total genes (782): ', np.round((52/782)*100,2),'%')

source_data_add='../results/Figs_Source_Data.xlsx'
if 0:
    saveAsNewSheetToExistingFile(source_data_add,table1,'ExtendedData6')

# gene_cats_bpcc.to_csv('./GO/go_bp_cc_D2021_each_gene_cat_completed.csv',index=False)
# gene_cats_bpcc.to_csv('./source/GO_bp_cc_mf_direct_intersection_782_completed.csv',index=False)


In [ ]:
table1['odds ratio'].astype(float).round(2)

In [ ]:
table1['odds ratio'].astype(float).round(2).values